In [1]:
!lsusb

Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
Bus 001 Device 058: ID 0403:6001 Future Technology Devices International, Ltd FT232 Serial (UART) IC
Bus 001 Device 006: ID 8087:07dc Intel Corp. 
Bus 001 Device 004: ID 1199:9041 Sierra Wireless, Inc. 
Bus 001 Device 009: ID 0b97:7772 O2 Micro, Inc. OZ776 CCID Smartcard Reader
Bus 001 Device 008: ID 0b97:7761 O2 Micro, Inc. Oz776 1.1 Hub
Bus 001 Device 007: ID 138a:0050 Validity Sensors, Inc. Swipe Fingerprint Sensor
Bus 001 Device 005: ID 04f2:b413 Chicony Electronics Co., Ltd 
Bus 001 Device 003: ID 04b4:6570 Cypress Semiconductor Corp. 
Bus 001 Device 002: ID 046d:c534 Logitech, Inc. Unifying Receiver
Bus 001 Device 001: ID 1d6b:0002 Linux Foundation 2.0 root hub


In [2]:
!ls -l /dev/ttyUSB*

crw-rw-rw- 1 root dialout 188, 0 Sep 20 15:21 /dev/ttyUSB0


In [3]:
!cat /etc/udev/rules.d/55-usb_serial.rules

SUBSYSTEMS=="usb", ATTRS{idVendor}=="067b", ATTRS{idProduct}=="2303", MODE:="0666"


In [1]:
import serial

## read the configuration registers

In [2]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
ser.write(b'\xC1\x00\x05')     # write a string
# read the response
response = ser.read(size=10)
print(response.hex())
ser.close()             # close port

/dev/ttyUSB0
0.5



## write the configuration (persistent)

In [16]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
# address = 00 00
# mode = 64 : 9600,8N1 air9.6k
# reg1 = 00
# freq = 12 : ch18=868.125 MHz
ser.write(b'\xC0\x00\x05\x00\x00\x64\x00\x12')     # command
# read the response
response = ser.read(size=10)
print(response.hex())
ser.close()             # close port

/dev/ttyUSB0
0.5



## write to one module, read from the other
(pull M0,M1 low for normal transmission mode)

In [4]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
ser.write(b'hello!')     # write a string
# read the response
response = ser.read(size=10)
print(response.hex(':'))
print(response)
ser.close()   

/dev/ttyUSB0
0.5


TypeError: hex() takes no arguments (1 given)

## receive and print
(pull M0,M1 low for normal transmission mode)

In [ ]:
# the groundstation modems are configured for a serial rate of 115200 baud
# over-the-air rate is still 9600 baud
# there is one additional character received - the RSI
ser = serial.Serial('/dev/ttyUSB0', baudrate=115200, timeout=0.1)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
print()
list_of_messages = []
stream_log = b''
global_timeout = 0
# we stop when we have nothing received for 100 cycles á 0.1s
while global_timeout<100:
    global_timeout += 1
    # read the response
    response = ser.read(size=200)
    stream_log += response
    if len(response) > 0:
        list_of_messages.append(response)
        line = ""
        for c in response:
            line += (" %0.2X" % c)
        print(line)
        print(response)
        print()
        global_timeout = 0

ser.close()
print('receiver stopped.')

/dev/ttyUSB0
0.1

 00 00
b'\x00\x00'

 70 69 6E 67 2E 0D 0A EC
b'ping.\r\n\xec'

 CC 82 08 47 50 53 5F 31 20 20 20 EE
b'\xcc\x82\x08GPS_1   \xee'

 70 69 6E 67 2E 0D 0A EF
b'ping.\r\n\xef'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 CC 82 08 47 50 53 5F 31 20 20 20 EE
b'\xcc\x82\x08GPS_1   \xee'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 CC 82 08 47 50 53 5F 31 20 20 20 EE
b'\xcc\x82\x08GPS_1   \xee'

 70 69 6E 67 2E 0D 0A EF
b'ping.\r\n\xef'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 CC 82 08 47 50 53 5F 31 20 20 20 EE
b'\xcc\x82\x08GPS_1   \xee'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 CC 82 08 47 50 53 5F 31 20 20 20 EE
b'\xcc\x82\x08GPS_1   \xee'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 CC 82 08 47 50 53 5F 31 20 20 20 EE
b'\xcc\x82\x08GPS_1   \xee'

 70
b'p'

 69 6E 67 2E 0D 0A EE
b'ing.\r\n\xee'

 70 69 6E 67 2E 0D 0A EE
b'ping.\r\n\xee'

 

 CC 82 08 47 50 53 5F 31 20 20 20 F0
b'\xcc\x82\x08GPS_1   \xf0'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 CC 3C 00 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xcc<\x00\x00systick overrun !\xf0'

 CC 81 2A 49 4D 55 5F 31 20 20 20 0C BE 3D 00 00 49 4D 55 20 6C 6F 6F 70 20 65 78 63 65 65 64 69 6E 67 20 31 30 20 63 79 63 6C 65 73 2E F0
b'\xcc\x81*IMU_1   \x0c\xbe=\x00\x00IMU loop exceeding 10 cycles.\xf0'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 B7 40 00 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xb7@\x00\x00systick overrun !\xf0'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 A2 44 00 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xa2D\x00\x00systick overrun !\xf0'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 8D 48 00 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\x8dH\x00\x00systick overrun !\xf0'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 78 4C 00 00 73 79 73 7

 CC 81 1E 53 59 53 54 45 4D 20 20 03 E1 CD 00 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xe1\xcd\x00\x00systick overrun !\xf0'

 CC 81 24 53 59 53 54 45 4D 20 20 03 CF D1 00 00 53 59 53 4C 4F 47 46 20 72 75 6E 74 69 6D 65 20 35 35 34 37 20 75 73 F0
b'\xcc\x81$SYSTEM  \x03\xcf\xd1\x00\x00SYSLOGF runtime 5547 us\xf0'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 CF D1 00 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xcf\xd1\x00\x00systick overrun !\xf0'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 BA D5 00 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xba\xd5\x00\x00systick overrun !\xf0'

 CC 82 08 47 50 53 5F 31 20 20 20 EF
b'\xcc\x82\x08GPS_1   \xef'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 A6 D9 00 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xa6\xd9\x00\x00systick overrun !\xf0'

 CC 81 1E 53 59 53 54 45
b'\xcc\x81\x1eSYSTE'

 4D 20 20 03 91 DD 00

 CC 82 08 47 50 53 5F 31 20 20 20 EF
b'\xcc\x82\x08GPS_1   \xef'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 C8 62 01 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 EF
b'\xcc\x81\x1eSYSTEM  \x03\xc8b\x01\x00systick overrun !\xef'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 B3 66 01 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 EF
b'\xcc\x81\x1eSYSTEM  \x03\xb3f\x01\x00systick overrun !\xef'

 CC 81 2A 49 4D 55 5F 31 20 20 20 0C BD 66 01 00 49 4D 55 20 6C 6F 6F 70 20 65 78 63 65 65 64 69 6E 67 20 31 30 20 63 79 63 6C 65 73 2E EF
b'\xcc\x81*IMU_1   \x0c\xbdf\x01\x00IMU loop exceeding 10 cycles.\xef'

 CC 81 1E 53 59
b'\xcc\x81\x1eSY'

 53 54 45 4D 20 20 03 9F 6A 01 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 EF
b'STEM  \x03\x9fj\x01\x00systick overrun !\xef'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 8A 6E 01 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 EF
b'\xcc\x81\x1eSYSTEM  \x03\x8an\x01\x00systick overrun !\xef'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 75 72 01 00 73 7

 CC 81 1E 53 59 53 54 45 4D 20 20 03 D9 F3 01 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xd9\xf3\x01\x00systick overrun !\xf0'

 CC 81 2A 49 4D 55 5F 31 20 20 20 0C D5 F6 01 00 49 4D 55 20 6C 6F 6F 70 20 65 78 63 65 65 64 69 6E 67 20 31 30 20 63 79 63 6C 65 73 2E EF
b'\xcc\x81*IMU_1   \x0c\xd5\xf6\x01\x00IMU loop exceeding 10 cycles.\xef'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 C4 F7 01 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 F0
b'\xcc\x81\x1eSYSTEM  \x03\xc4\xf7\x01\x00systick overrun !\xf0'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 AF FB 01 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 EF CC 82
b'\xcc\x81\x1eSYSTEM  \x03\xaf\xfb\x01\x00systick overrun !\xef\xcc\x82'

 08 47 50 53 5F 31 20 20 20 EF
b'\x08GPS_1   \xef'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 9A FF 01 00 73 79 73 74 69 63 6B 20 6F 76 65 72 72 75 6E 20 21 EF
b'\xcc\x81\x1eSYSTEM  \x03\x9a\xff\x01\x00systick overrun !\xef'

 CC 81 1E 53 59 53 54 45 4D 20 20 03 85 03 0

In [8]:
msg=list_of_messages[6]
msg

b'\xcc\x81\x19DISPLAY \n\x90\x05\x00\x00initialized.\xf3'

### write the logged stream to file

In [41]:
print(stream_log)

b'TAROS : MODEM_1 up and running.\r\n\xfd\xcc\x813SYSTEM  \x05\x00\x00\x00\x00Teensy Flight Controller - Version 1.0\xfd\xcc\x81\x1cUSB_1   \x05\x00\x00\x00\x00up and running.\xfd\xcc\x81\x1cSYSTEM  \x05\x00\x00\x00\x00build complete.\xfd\xcc\x81!SYSTEM  \x05\x00\x00\x00\x00entering event loop.\xfd\xcc\x81\x1cGPS_1   \x05\x91\x01\x00\x00up and running.\xfd\xcc\x81\x1eSYSTEM  \x03\x91\x01\x00\x00systick overrun !\xfd\xcc\x81\x19DISPLAY \n\x88\x05\x00\x00initialized.\xfd\xcc\x81\x1eSYSTEM  \x03\x88\x05\x00\x00systick overrun !\xfdping\r\n\xfd\xcc\x82\x08GPS_1   \xfc\xcc\x81\x1bGPS_1   \n\x89\x13\x00\x00acquired lock.\xfdping\r\n\xfdping\r\n\xfc\xcc\x82\x08GPS_1   \xfcping\r\n\xf3'


In [45]:
f = open('stream_log.dat', 'wb')
f.write(stream_log)
f.close()

### handle and decode messages

from struct import *

msb, lsb, n_bytes, level = unpack('BBBxxxxxxxxB', msg[:12])
sender = msg[3:11].decode(encoding='utf-8')
time, = unpack('I', msg[12:16])
text = msg[16:n_bytes+3].decode(encoding='utf-8')
rsi = msg[n_bytes+3]
print(msb)
print(lsb)
print(n_bytes)
print(level)
print(time)
print(sender)
print(text)
print(rsi)

In [12]:
from struct import *
import pandas as pd

In [13]:
def format_time(t):
    h = t//(1000*60*60)
    ms = t-h*(1000*60*60)
    m = ms//(1000*60)
    ms = ms-m*(1000*60)
    s = ms//1000
    ms = ms-s*1000
    return "%02d:%02d:%02d.%03d" % (h,m,s,ms)

In [14]:
sys_msg_list = []
for msg in list_of_messages:
    if len(msg)>=3:
        msb, lsb, n_bytes = unpack('BBB', msg[:3])
        # if it ihas a message header
        if msb == 204:
            # if it is a system message
            if lsb == 129:
                sender = msg[3:11].decode(encoding='utf-8')
                level = msg[11]
                time, = unpack('I', msg[12:16])
                text = msg[16:n_bytes+3].decode(encoding='utf-8')
                rsi = msg[n_bytes+3]
                print("%3d"%level, sender, format_time(time), text)
                sys_msg_list.append([sender,level,time,text,rsi])

  5 SYSTEM   00:00:00.000 Teensy Flight Controller - Version 1.0
  5 SYSTEM   00:00:00.000 found SD card.
  5 USB_1    00:00:00.000 up and running.
  5 SYSTEM   00:00:00.000 build complete.
  5 SYSLOGF  00:00:00.010 file opened.
 10 DISPLAY  00:00:01.424 initialized.
 10 IMU_1    00:00:01.424 BNO-055 setup()
 10 IMU_1    00:00:02.161 BNO-055 begin() success.
 10 IMU_1    00:00:02.213 BNO-055 initialized.
 10 IMU_1    00:00:02.271 BNO-055 calibrated.
  5 IMU_1    00:00:02.321 up and running.
  5 SYSTEM   00:00:02.698 entering event loop.
  5 GPS_1    00:00:02.698 up and running.
  3 SYSTEM   00:00:02.701 systick overrun !
  3 SYSTEM   00:00:02.711 SYSLOGF runtime 10253 µs

  3 SYSTEM   00:00:02.803 MODEM_1 runtime 1881 µs

  3 SYSTEM   00:00:02.876 SYSLOGF runtime 72231 µs

  3 SYSTEM   00:00:02.876 systick overrun !
  3 SYSTEM   00:00:02.932 MODEM_1 runtime 1361 µs

  3 SYSTEM   00:00:02.934 SYSLOGF runtime 1374 µs

  3 SYSTEM   00:00:03.056 USB_1 runtime 120988 µs

  3 SYSTEM   00:00:

IndexError: index out of range

In [15]:
df = pd.DataFrame(sys_msg_list, columns = ['sender', 'level', 'time', 'text', 'RSI'])
df

,sender,level,time,text,RSI
0,SYSTEM,5,0,Teensy Flight Controller - Version 1.0,243
1,SYSTEM,5,0,found SD card.,243
2,USB_1,5,0,up and running.,243
3,SYSTEM,5,0,build complete.,243
4,SYSLOGF,5,10,file opened.,243
5,DISPLAY,10,1424,initialized.,243
6,IMU_1,10,1424,BNO-055 setup(),243
7,IMU_1,10,2161,BNO-055 begin() success.,243
8,IMU_1,10,2213,BNO-055 initialized.,243
9,IMU_1,10,2271,BNO-055 calibrated.,243


In [16]:
text_msg_list = []
for msg in list_of_messages:
    if len(msg)>=3:
        msb, lsb, n_bytes = unpack('BBB', msg[:3])
        # if it has a message header
        if msb == 204:
            # if it is a text message
            if lsb == 130:
                sender = msg[3:11].decode(encoding='utf-8')
                text = msg[11:n_bytes+3].decode(encoding='utf-8')
                rsi = msg[n_bytes+3]
                text_msg_list.append([sender, text, rsi])
df = pd.DataFrame(text_msg_list, columns = ['sender', 'text', 'RSI'])
df

,sender,text,RSI
0,GPS_1,,243
1,GPS_1,,243


### parse messages from stream

In [67]:
f = open('stream_log.dat', 'rb')
buffer = f.read()
f.close()

print(len(buffer))
print(buffer[20])
print(buffer)

402
110
b'TAROS : MODEM_1 up and running.\r\n\xfd\xcc\x813SYSTEM  \x05\x00\x00\x00\x00Teensy Flight Controller - Version 1.0\xfd\xcc\x81\x1cUSB_1   \x05\x00\x00\x00\x00up and running.\xfd\xcc\x81\x1cSYSTEM  \x05\x00\x00\x00\x00build complete.\xfd\xcc\x81!SYSTEM  \x05\x00\x00\x00\x00entering event loop.\xfd\xcc\x81\x1cGPS_1   \x05\x91\x01\x00\x00up and running.\xfd\xcc\x81\x1eSYSTEM  \x03\x91\x01\x00\x00systick overrun !\xfd\xcc\x81\x19DISPLAY \n\x88\x05\x00\x00initialized.\xfd\xcc\x81\x1eSYSTEM  \x03\x88\x05\x00\x00systick overrun !\xfdping\r\n\xfd\xcc\x82\x08GPS_1   \xfc\xcc\x81\x1bGPS_1   \n\x89\x13\x00\x00acquired lock.\xfdping\r\n\xfdping\r\n\xfc\xcc\x82\x08GPS_1   \xfcping\r\n\xf3'


In [68]:
print(b'\xcc')

b'\xcc'


In [6]:
from struct import *


In [9]:

# in real world the buffer will be refilled with every received message
f = open('stream_log.dat', 'rb')
buffer = f.read()
f.close()
print(len(buffer))

sys_msg_list = []
text_msg_list = []

while len(buffer) > 0:
    next_index = 0
    byte = buffer[0]
    if byte != 0xcc:
        # skip
        buffer = buffer[1:]
        continue
    # here we have a potential message start
    byte = buffer[1]
    if byte == 0x81:
        size = buffer[2]
        print("system message, length=%d" % size)
        sender = buffer[3:11].decode(encoding='utf-8')
        level = buffer[11]
        time, = unpack('I', buffer[12:16])
        text = buffer[16:size+3].decode(encoding='utf-8')
        rsi = buffer[size+3]
        print("%3d"%level, sender, format_time(time), text)
        sys_msg_list.append([sender,level,time,text,rsi])
        next_index = 3 + size + 1
    elif byte == 0x82:
        size = buffer[2]
        print("text message, length=%d" % size)
        sender = buffer[3:11].decode(encoding='utf-8')
        text = buffer[11:size+3].decode(encoding='utf-8')
        rsi = buffer[size+3]
        print(sender, text)
        text_msg_list.append([sender, text, rsi])
        next_index = 3 + size + 1
    else:
        # unknown message type
        next_index = 1
    # remove the handled bytes from the buffer
    buffer = buffer[next_index:]
    

402
system message, length=51
  5 SYSTEM   00:00:00.000 Teensy Flight Controller - Version 1.0
system message, length=28
  5 USB_1    00:00:00.000 up and running.
system message, length=28
  5 SYSTEM   00:00:00.000 build complete.
system message, length=33
  5 SYSTEM   00:00:00.000 entering event loop.
system message, length=28
  5 GPS_1    00:00:00.401 up and running.
system message, length=30
  3 SYSTEM   00:00:00.401 systick overrun !
system message, length=25
 10 DISPLAY  00:00:01.416 initialized.
system message, length=30
  3 SYSTEM   00:00:01.416 systick overrun !
text message, length=8
GPS_1    
system message, length=27
 10 GPS_1    00:00:05.001 acquired lock.
text message, length=8
GPS_1    


In [92]:
df = pd.DataFrame(sys_msg_list, columns = ['sender', 'level', 'time', 'text', 'RSI'])
df

,sender,level,time,text,RSI
0,SYSTEM,5,0,Teensy Flight Controller - Version 1.0,253
1,USB_1,5,0,up and running.,253
2,SYSTEM,5,0,build complete.,253
3,SYSTEM,5,0,entering event loop.,253
4,GPS_1,5,401,up and running.,253
5,SYSTEM,3,401,systick overrun !,253
6,DISPLAY,10,1416,initialized.,253
7,SYSTEM,3,1416,systick overrun !,253
8,GPS_1,10,5001,acquired lock.,253


In [90]:
df = pd.DataFrame(text_msg_list, columns = ['sender', 'text', 'RSI'])
df

,sender,text,RSI
0,GPS_1,,252
1,GPS_1,,252
